In [1]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from utils.utils import pretty_lat, pretty_lon, ProgressStatus
from tempfile import TemporaryDirectory

In [2]:
from station.data_submission import DataSubmission
from station.station import StationData

from train_station_twin.training_executer import TrainingExecuter
from infilling.evaluation_executer import EvaluatuionExecuter
from infilling.infilling_writer import InfillingWriter

from era5.era5_for_station import DownloadEra5ForStation, DownloadEra5ForStationGaps
from era5.era5_download_hook import Era5DownloadHook


In [3]:
def plot_n_steps_of_area_from_nc_file(path, n=1, vars="tas", title="", vmin=None, vmax=None):

    dataset = xr.open_dataset(path)

    n = min(n, dataset.time.size)
    time_index_list = np.random.choice(dataset.time.size, n, replace=False)

    lat_slice = slice(None)
    lon_slice = slice(None)
    _lon = dataset.lon.values[lon_slice]
    _lat = dataset.lat.values[lat_slice]

    # if not list make it a list
    if not isinstance(vars, list):
        vars = [vars]

    for time_index in time_index_list:
        # set title
        title += f"\n{dataset.time.values[time_index].astype('datetime64[s]').astype('O')}"

        # subtitle lat lon area
        subtitle = f"\nLat: {pretty_lat(_lat[0])} to {pretty_lat(_lat[-1])}" + \
            f"\nLon: {pretty_lon(_lon[0])} to {pretty_lon(_lon[-1])}"

        for var in vars:

            # plot
            fig, ax = plt.subplots(
                subplot_kw={'projection': ccrs.PlateCarree()})
            # Plot the temperature data with a quadratic colormap
            _data = dataset.variables[var].values[time_index, lat_slice, lon_slice]
            pcm = ax.pcolormesh(_lon, _lat, _data, cmap='viridis',
                                shading='auto', vmin=vmin, vmax=vmax)

            # Add coastlines
            ax.coastlines()

            # Add colorbar
            cbar = plt.colorbar(pcm, ax=ax, label='Temperature')

            # Set labels and title
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            plt.title(title + (f"\n[{var}]" if len(vars) > 1 else ""))

            # position title a higher
            plt.subplots_adjust(top=1)
            
            # Add subtitle
            plt.figtext(0.125, 0.05, subtitle, wrap=True, horizontalalignment='left', fontsize=12)

            # Show the plot
            plt.show()

In [4]:
marshall_data = DataSubmission("Marshall_2017-2023")
marshall_data.measurement_dir_path = "./measurements/Marshall/"

marshall_station = StationData(
    name=marshall_data.name,
    folder_path=marshall_data.measurement_dir_path
)

Extracting Marshall_2017-2023...


100%|██████████| 306/306 [00:19<00:00, 16.01it/s]


In [5]:

training = TrainingExecuter(station=marshall_station, ProgressStatus())

output_model_path = training.execute()
marshall_data.add_model(output_model_path)



(7177,)
Saving to /var/folders/yj/k7pg73d56rl366r8bvlbncgm0000gn/T/tmpxrz6o9jp/train/marshall_2017-2023.nc


2024-04-15 23:41:25,084 INFO Welcome to the CDS
2024-04-15 23:41:25,085 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-04-15 23:41:25,196 INFO Request is completed
2024-04-15 23:41:25,197 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data2/adaptor.mars.internal-1713205445.4090545-24831-2-52ca6b2c-1219-4190-b653-f4db21242676.grib to /var/folders/yj/k7pg73d56rl366r8bvlbncgm0000gn/T/tmpr9nyfy9_/2017.grib (3M)
2024-04-15 23:41:25,873 INFO Download rate 4.5M/s   


Found 2017.grib
Renamed variable var167 to tas in /var/folders/yj/k7pg73d56rl366r8bvlbncgm0000gn/T/tmpf0irwh95/2017.nc
Found 2017.nc
Merged era5 file saved into /var/folders/yj/k7pg73d56rl366r8bvlbncgm0000gn/T/tmpm7d1y6d5/era5_merged.nc
Lat:  39.9496
Lon:  254.804
nearest_lat_idx: 4 [40.95     40.699875 40.44975  40.199625 39.9495   39.699375 39.44925
 39.199125 38.949   ]
nearest_lon_idx: 4 [253.804 254.054 254.304 254.554 254.804 255.054 255.304 255.554 255.804]
nearest_lat: 39.9495
nearest_lon: 254.804
nearest lon is bigger
nearest lat is smaller
crop_lon_idx_min: 0
crop_lon_idx_max: 7
crop_lat_idx_min: 0
crop_lat_idx_max: 7
Lon values after cropping: [253.804 254.054 254.304 254.554 254.804 255.054 255.304 255.554]
Lat values after cropping: [40.95     40.699875 40.44975  40.199625 39.9495   39.699375 39.44925
 39.199125]
train args: --data-root-dir /var/folders/yj/k7pg73d56rl366r8bvlbncgm0000gn/T/tmpxrz6o9jp
            --data-names era5_merged.nc,cleaned.nc
            --data-typ

usage: ipykernel_launcher.py [-h] [--data-root-dir DATA_ROOT_DIR]
                             [--mask-dir MASK_DIR] [--log-dir LOG_DIR]
                             [--data-names DATA_NAMES]
                             [--mask-names MASK_NAMES]
                             [--data-types DATA_TYPES]
                             [--n-target-data N_TARGET_DATA] [--device DEVICE]
                             [--shuffle-masks] [--channel-steps CHANNEL_STEPS]
                             [--lstm-steps LSTM_STEPS] [--gru-steps GRU_STEPS]
                             [--encoding-layers ENCODING_LAYERS]
                             [--pooling-layers POOLING_LAYERS]
                             [--conv-factor CONV_FACTOR] [--weights WEIGHTS]
                             [--steady-masks STEADY_MASKS]
                             [--loop-random-seed LOOP_RANDOM_SEED]
                             [--cuda-random-seed CUDA_RANDOM_SEED]
                             [--deterministic] [--attention]
  

SystemExit: 2

/Users/timo.wacke/anaconda3/envs/crai/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
barbados_station = StationData("Barbados", "./measurements/Barbados/")
vienna_station = StationData("Vienna", "./measurements/Vienna/")

In [1]:
import asyncio
import pty
import os

async def run_script():
    # Path to your Python script
    script_path = "test.py"
    
    # Initialize counter for the number of lines found
    line_count = 0
    
    # Start the subprocess with pty
    master, slave = pty.openpty()
    process = await asyncio.create_subprocess_exec(
        "python", script_path,
        stdin=slave,
        stdout=slave,
        stderr=slave
    )
    
    try:
        # print the length of the output every 5 seconds
        while True:
            output = os.read(master, 1024).decode()
            if not output:
                # If there's no more output, break out of the loop
                print("breaaak")
                break
            
            line_count += len(output.splitlines())
            print(f"Lines found: {line_count}")
            await asyncio.sleep(5)
    except OSError as e:
        print(f"Error reading output: {e}")
    finally:
        # Close the file descriptors
        os.close(master)
        os.close(slave)

# Create a task to run the coroutine in the background
task = asyncio.create_task(run_script())

# Wait for the task to complete
await task


1
2
